# Assistant de recherche avec Gemini, LangChain, and Chroma

## Objectif

L'objectif principal de ce workshop est de concevoir un assistant de recherche dédié au traitement de papiers scientifiques. Cet outil permettra d'importer et d'indexer ces documents dans le but de faciliter la réponse à des questions de recherche ultérieures.

## Installation des packages nécessaires

In [ ]:
!pip install -U -q langchain-community
!pip install -q langchain-google-genai
!pip install -q chromadb
!pip install -q pypdf

### Génération de Gemeni API Key

Pour utiliser les modèles Gemini de Google via une API, il est nécessaire d'obtenir une clé API. Cette clé peut être générée facilement et gratuitement via l'interface de Google AI Studio  [Google AI Studio](https://makersuite.google.com/).

In [13]:
import os
import getpass

os.environ['GOOGLE_API_KEY'] = "XXXXXXXXXXXX" # Ici remplacez par votre API Key !!!

## Import des packages nécessaires

In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
import getpass
from langchain import PromptTemplate
from langchain import hub
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Retriever

Cette étape, nommée "Retriever", consiste à préparer des pdf pour un modèle linguistique (LLM).

1. Lire les fichiers pdf avec LangChain.
2. Créer des embeddings (représentations numériques vectorielles) de ces données en utilisant le modèle de Gemini. Les textes similaires auront des vecteurs similaires.
3. Stocker ces embeddings dans Chroma, une base de données vectorielle, pour les retrouver facilement.
4. Créer un "Retriever" à partir de Chroma pour récupérer les informations pertinentes (embeddings) à passer au LLM en fonction des questions de l'utilisateur.

### Chargement des fichiers pdf

In [6]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def get_file_text(file_path):
  loader = PyPDFLoader(file_path)
  pages = []
  for page in loader.load():
    pages.append(page)

  return format_docs(pages)

In [9]:
final_text = get_file_text(r"ici il faut remplacer par le path de votre fichier .pdf")
#print(final_text)

### Initialisation du modèle d'embedding (modèle de Gemini)

In [15]:
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

### Embedding et sauvegarde des données


In [16]:
def load_file(file_path):
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    return pages

def generate_vectorstore(file_path):
    pages = load_file(file_path)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )

    chunked_documents = text_splitter.split_documents(pages)

    print(f"Chargé {len(pages)} pages et créé {len(chunked_documents)} chunks.")

    vectorstore = Chroma.from_documents(
                        documents=chunked_documents,
                        embedding=gemini_embeddings,
                        persist_directory="./chroma_db"
                    )

    print(f"Base de données vectorielle créée/mise à jour dans ./chroma_db avec {len(chunked_documents)} chunks.")

In [17]:
generate_vectorstore(r"ici il faut remplacer par le path de votre fichier .pdf")

Chargé 599 pages et créé 992 chunks.
Base de données vectorielle créée/mise à jour dans ./chroma_db avec 992 chunks.


### Creation d'un "retriever" avec Chroma

In [18]:
def get_retriver(gemini_embeddings):
  vectorstore_disk = Chroma(
                        persist_directory="./chroma_db",
                        embedding_function=gemini_embeddings
                   )
  return vectorstore_disk.as_retriever(search_kwargs={"k": 3})

In [19]:
retriever = get_retriver(gemini_embeddings)

<ipython-input-18-4f3f5f1247b2>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore_disk = Chroma(


## Génération de réponses

L'étape de génération de réponses consiste à demander une réponse au LLM lorsque l'utilisateur pose une question. Le "retriever", créé précédemment à partir de la base de données vectorielle Chroma, fournira au LLM les informations pertinentes (les embeddings du texte) pour donner du contexte à la question de l'utilisateur.

Les étapes sont :

1. Combiner un prompt pour le retriever, un prompt pour la réponse, et le modèle LLM.
2. Exécuter cette combinaison avec la question de l'utilisateur pour obtenir une réponse du modèle.


### Initialisation du LLM Gemini

In [20]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7, top_p=0.85)

### Création du template de prompt

Dans le modèle de prompt, la variable "question" sera remplacée par la question de l'utilisateur, et la variable "context" par les paragraphes récupérés depuis Chroma.

In [21]:
def get_prompt():
  llm_prompt_template = """You are an assistant for question-answering tasks.
  Use the following context to answer the question.
  If you don't know the answer, just say that you don't know.
  Use five sentences maximum and keep the answer concise.\n
  Question: {question} \nContext: {context} \nAnswer:"""

  return PromptTemplate.from_template(llm_prompt_template)

In [22]:
llm_prompt = get_prompt()

### Creation de la chaine de réponse

In [23]:
def get_llm_response(question):
  retriever = get_retriver(gemini_embeddings)
  llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7, top_p=0.85)
  llm_prompt = get_prompt()
  rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | llm_prompt
      | llm
      | StrOutputParser()
  )

  return rag_chain.invoke(question)

### Test de la solution

In [28]:
get_llm_response("Give me a detailed definition of generative ai")

'Generative models generate sequences of tokens from sequences of tokens. These models are trained on a wide variety of tasks and usually do not perform your use case out of the box. To guide the model, you can use instructions or prompts. Generative models use autoregressive, token-by-token generation. They can also leverage a tree-based structure to generate intermediate thoughts.'